<a href="https://colab.research.google.com/github/zerryberry/C1/blob/master/others.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import xgboost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRFRegressor

In [ ]:
df= pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/cancer_death_rate/Training_set_label.csv')
dftest = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/cancer_death_rate/Testing_set_label.csv')
print(df.shape)
df.isna().any()

In [ ]:
# correlation time!

fig, ax= plt.subplots(figsize=(10,10))
sns.heatmap(df.corr())

In [ ]:
a= set()
for i in df.columns:
    for j in df.columns:
        if i in df.select_dtypes(include='number').columns and j in df.select_dtypes(include='number').columns and i!= j and df[i].corr(df[j])>0.7:
            a.add(i)

dfn = df[list(a)]
# dfn.isna().any()

In [ ]:
dfn = pd.concat((dfn, df['TARGET_deathRate']),axis=1)
dfn['TARGET_deathRate'].corr(df['PctEmployed16_Over'])

# as both of them has negative correlation (less) with the target, we can drop them. 

dfn.drop({'PctPrivateCoverageAlone', 'PctEmployed16_Over'}, axis=1, inplace= True)

In [ ]:
meann= []
mediann = []
for i in dfn.columns:
    meann.append(dfn[i].mean())
    mediann.append(dfn[i].median())

outliers= pd.DataFrame(meann, mediann)
outliers.set_index(dfn.columns, inplace=True)
outliers.rename({0: 'mean'},axis=1, inplace= True)
outliers['median'] = mediann

In [ ]:
# outlier database
outliers

In [ ]:
outliers.plot(kind='bar')

### population estimation is an obvious outlier. 

### another one is employee priv coverage, because of which the scale of other magnitude are sort of getting hid. 
### Removing these will help us see the actual scale. 

In [ ]:
outliers.iloc[(outliers.index != 'popEst2015') & (outliers.index != 'medIncome')].plot(kind='bar')

There are two more columns which are showing potential outliers as mean is greater than median. average annual count and average deaths per year.

In [ ]:
dfn['avgAnnCount'].plot(kind='box')

In [ ]:
dfn['avgDeathsPerYear'].corr(df['TARGET_deathRate'])

-0.07400827212442411

In [ ]:
# Since both of them has very less correlation with target, its very unlikely that outliers from these features are going to affect
# the target of the ML.

# rest of the data looks fine, so lets start with ML
std = StandardScaler()
x_std_tr= std.fit_transform(x_train)

In [ ]:
# normalization
x_std_test= std.fit_transform(x_test)

In [ ]:
cf= pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/cancer_death_rate/Training_set_label.csv')
cf.dtypes

In [ ]:
from sklearn.ensemble import RandomForestRegressor
ee= RandomForestRegressor().fit(x_train, y_train)
ee.score(x_train, y_train)

0.9490544466164118

In [ ]:
ee.score(x_test, y_test)

0.6241585597107988

In [ ]:
############################################################################################## All Variables

cf= pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/cancer_death_rate/Training_set_label.csv')
# cf.dtypes

In [ ]:
cf.drop({'binnedInc','Geography'}, axis=1, inplace=True)

In [ ]:
cf.isna().any()

In [ ]:
cf.drop({'PctSomeCol18_24','PctEmployed16_Over', 'PctPrivateCoverageAlone'}, axis=1, inplace=True)

In [ ]:
y= cf['TARGET_deathRate']
cf.drop({'TARGET_deathRate'}, axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_tr, x_t, y_tr, y_t = train_test_split(cf, y)

In [ ]:
#from sklearn.ensemble import RandomForestRegressor
ee= RandomForestRegressor().fit(x_tr, y_tr)
print(ee.score(x_tr, y_tr))
print(ee.score(x_t, y_t))

0.9533051946845945
0.6890818514839356


In [ ]:
########################################################################################################################3

In [ ]:

yy= ['incidenceRate', 'medIncome', 'povertyPercent', 'PctHS25_Over', 'PctBachDeg25_Over', 'PctEmployed16_Over', 'PctUnemployed16_Over', 'PctPrivateCoverage', 
    ]
new= nf[yy]
new.head()

,incidenceRate,medIncome,povertyPercent,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage
0,481.5,50038,11.1,37.1,12.2,57.4,1.1,81.2
1,486.0,59399,9.7,35.9,16.0,60.4,3.8,76.7
2,475.7,39721,18.5,40.2,7.6,41.2,11.0,61.6
3,496.6,30299,28.1,44.2,7.0,41.6,10.4,45.2
4,372.0,39625,17.4,28.3,10.5,49.3,9.2,56.6


In [ ]:
# nf[nf.columns[1:]].corr()['TARGET_deathRate'][:] 
y = nf['TARGET_deathRate']

In [ ]:
new['PctEmployed16_Over'].fillna(new['PctEmployed16_Over'].median(), inplace= True)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
scaler = preprocessing.StandardScaler() 
standard_df = scaler.fit_transform(new)

In [ ]:
# y_st= scaler.fit_transform([np.array(y)])

In [ ]:
u= pd.DataFrame(standard_df, columns= yy)

In [ ]:
u

In [ ]:
# u_tar= pd.Series(y_st.ravel())

In [ ]:
x_tr, x_t, y_tr, y_t = train_test_split(u, y)

In [ ]:
################################ SCORE #####################################################
ee= RandomForestRegressor().fit(x_tr, y_tr)
print(ee.score(x_tr, y_tr))
print(ee.score(x_t, y_t))

0.9531210040604357
0.6382736102187889


In [ ]:
# test

In [ ]:
tn= dftest[yy]
tn.head()

,incidenceRate,medIncome,povertyPercent,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage
0,341.000000,34368,33.5,23.2,11.9,52.2,9.9,32.6
1,445.200000,35615,21.5,47.1,7.9,46.5,9.0,55.6
2,425.700000,76672,7.6,30.5,17.5,71.4,2.8,83.3
3,453.549422,45856,11.4,35.3,15.5,63.1,4.3,75.8
4,453.549422,53739,10.9,29.7,15.5,67.9,2.4,78.2


In [ ]:
tn['PctEmployed16_Over'].fillna(tn['PctEmployed16_Over'].median(), inplace= True)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
scaler = preprocessing.StandardScaler() 
standard_df_t = scaler.fit_transform(tn)

In [ ]:
u_test =pd.DataFrame(standard_df_t, columns= yy)
u_test.head()

In [ ]:
ee.predict(tn)

In [ ]:
#######################################################################################

zf= pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/cancer_death_rate/Training_set_label.csv')

In [ ]:
zf.dtypes

In [ ]:
zf.isna().any()

In [ ]:
zf.drop({'PctPrivateCoverageAlone'}, axis=1, inplace= True)

In [ ]:
ty = zf

In [ ]:
y = ty['TARGET_deathRate']

In [ ]:
ty.drop({'TARGET_deathRate'}, axis=1, inplace= True)

In [ ]:
scaler = preprocessing.StandardScaler() 
sdf = scaler.fit_transform(ty)

In [ ]:
urd = pd.DataFrame(sdf, columns= ty.columns)

In [ ]:
x_tr, x_t, y_tr, y_t = train_test_split(urd, y)

In [ ]:
ee= RandomForestRegressor(n_estimators=300).fit(x_tr, y_tr)
print(ee.score(x_tr, y_tr))
print(ee.score(x_t, y_t))

0.9570530893803436
0.6843274061595293


In [ ]:
ff= dftest[ty.columns]

In [ ]:
scaler = preprocessing.StandardScaler() 
sdf2 = scaler.fit_transform(ff)

In [ ]:
urd2 = pd.DataFrame(sdf2, columns= ff.columns)

In [ ]:
preds= ee.predict(urd2)

In [ ]:
preds1 = pd.DataFrame(preds, columns=['prediction'], index= None)

In [ ]:
preds1.set_index('prediction', inplace= True)

In [ ]:
preds1.to_csv('preds.csv')

In [ ]:
####################
df_01 = df.copy()
df_0t = dftest.copy()

In [ ]:
df_01.isna().any()

In [ ]:
y= ['PctSomeCol18_24', 'PctEmployed16_Over', 'PctPrivateCoverageAlone']
df_01.drop(y, axis=1, inplace= True)

In [ ]:
y1= df_01['TARGET_deathRate']
df_01.drop({'TARGET_deathRate'}, axis=1, inplace= True)
df_01.head()

In [ ]:
df_01['binnedInc'] = df_01['binnedInc'].astype('category')

In [ ]:
dftest['binnedInc'] = dftest['binnedInc'].astype('category')

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cat_ = label_encoder.fit_transform(df_01['binnedInc'])

In [ ]:
cat_test = label_encoder.fit_transform(dftest['binnedInc'])

In [ ]:
cat_

array([5, 7, 1, ..., 6, 2, 9])

In [ ]:
dftest['binnedInc'] = cat_test

In [ ]:
df_01['binnedInc'][:10]

In [ ]:
df_01['binnedInc'] = cat_

In [ ]:
type(df_01['binnedInc'])

pandas.core.series.Series

In [ ]:
df_01.select_dtypes(include='object')

In [ ]:
dftest.dtypes

In [ ]:
cat_1 = label_encoder.fit_transform(df_01['Geography'])
df_01['Geography'] = cat_1

In [ ]:
cat_22= label_encoder.transform(dftest['Geography'])
dftest['Geography'] = cat_22

In [ ]:
pre_ = sklearn.preprocessing.StandardScaler()
df_trans= pre_.fit_transform(df_01)

In [ ]:
df_trans_test = pre_.transform(dftest)

array([[-0.44121396, -0.37755589,  0.61449244, ..., -0.5765122 ,
         1.70996348,  4.8843226 ],
       [-0.38602904, -0.31912496,  0.69959445, ..., -0.56403137,
         1.57654428, -1.18439006],
       [-0.30045242, -0.20900514,  0.50480541, ..., -0.55105377,
         0.40194789,  0.39975724],
       ...,
       [ 1.11329541, -0.37980323,  0.08590349, ..., -0.10429711,
         0.8949667 ,  2.10565733],
       [-0.38842838, -0.28766216, -1.2331667 , ..., -0.49645919,
         0.10805446,  0.34496379],
       [-0.41242182, -0.33485637, -2.03123442, ..., -0.41037812,
        -0.08673052,  2.27026577]])

In [ ]:
dftest.shape

(762, 33)

In [ ]:
######################################################################################################################

In [2]:
tf= pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/cancer_death_rate/Training_set_label.csv')
tftest = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/cancer_death_rate/Testing_set_label.csv')

In [ ]:
tf.select_dtypes(include='object')

In [ ]:
tf['PctSomeCol18_24'].corr(tf['TARGET_deathRate'])

-0.2063373178462027

In [ ]:
# drop PctSomeCol18_24
# nan handling PctEmployed16_Over, PctPrivateCoverageAlone
# cate var binnedInc, Geography
# standard scaler

In [ ]:
tf.head()

In [3]:
y_ = tf['TARGET_deathRate']
tf.drop({'TARGET_deathRate'}, axis=1, inplace = True)

In [4]:
tf.drop({'PctSomeCol18_24'}, axis=1, inplace = True)
tftest.drop({'PctSomeCol18_24'}, axis=1, inplace = True)

In [5]:
tf['PctEmployed16_Over'].fillna(tf['PctEmployed16_Over'].mean(), inplace = True)

In [6]:
tftest['PctEmployed16_Over'].fillna(tftest['PctEmployed16_Over'].mean(), inplace = True)

In [7]:
tf['PctPrivateCoverageAlone'].fillna(tf['PctPrivateCoverageAlone'].mean(), inplace = True)

In [8]:
tftest['PctPrivateCoverageAlone'].fillna(tftest['PctPrivateCoverageAlone'].mean(), inplace = True)

In [9]:
tf['binnedInc'] = tf['binnedInc'].astype('category')
tftest['binnedInc'] = tftest['binnedInc'].astype('category')
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cat_1 = label_encoder.fit_transform(tf['binnedInc'])
tf['binnedInc'] = cat_1
cat_test = label_encoder.transform(tftest['binnedInc'])
tftest['binnedInc'] = cat_test

In [10]:
tf['Geo_cat'] = [tf['Geography'][i].split(',')[1].strip() for i in range(len(tf['Geography']))]

In [11]:
tftest['Geo_cat'] = [tftest['Geography'][i].split(',')[1].strip() for i in range(len(tftest['Geography']))]

In [ ]:
len(tf['Geo_cat'].unique())

51

In [ ]:
len(tftest['Geo_cat'].unique())

50

In [ ]:
set(tf['Geo_cat'].unique()).difference(set(tftest['Geo_cat'].unique()))

{'District of Columbia'}

In [ ]:
tf.loc[tf['Geo_cat']== 'District of Columbia']

In [12]:
tf['Geo_cat'].replace({'District of Columbia': 'Washington'}, inplace=True)

In [ ]:
# empty set
set(tf['Geo_cat'].unique()).difference(set(tftest['Geo_cat'].unique()))

In [14]:
tf['Geo_cat'] = tf['Geo_cat'].astype('category')
tftest['Geo_cat'] = tftest['Geo_cat'].astype('category')
label_encoder_geo = LabelEncoder()
cat_1 = label_encoder_geo.fit_transform(tf['Geo_cat'])
tf['Geo_cat'] = cat_1
cat_test = label_encoder_geo.transform(tftest['Geo_cat'])
tftest['Geo_cat'] = cat_test

In [15]:
tf.drop({'Geography'}, axis=1, inplace= True)

In [ ]:
tftest.drop({'Geography'}, axis=1, inplace= True)
tftest.select_dtypes(include='object')

In [17]:
print(tf.shape)
print(tftest.shape)

(3051, 32)
(762, 32)


In [52]:
from sklearn.preprocessing import StandardScaler
tysc = StandardScaler()
tf_sc= tysc.fit_transform(tf)
tftest_sc = tysc.transform(tftest)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
x_train, x_test, y_train, y_test = train_test_split(tf_sc, y_, random_state=42)

lr= LinearRegression().fit(x_train, y_train)

In [ ]:
print(lr.score(x_train, y_train))
print(lr.score(x_test, y_test))

0.5282041585775187
0.5423184300484032


In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# lrr= RandomForestRegressor(max_depth=18, max_features=12).fit(x_train, y_train)
print(lrr.score(x_train, y_train))
print(lrr.score(x_test, y_test))

0.955745796534607
0.7079168853269004


In [ ]:
#from sklearn.ensemble import GradientBoostingRegressor
# gbr = GradientBoostingRegressor(learning_rate=0.5, n_estimators=150, max_depth=3).fit(x_train, y_train)
print(gbr.score(x_train, y_train))
print(gbr.score(x_test, y_test))

0.9743875032737885
0.7454620978142507


In [ ]:
xg = XGBRegressor(learning_rate=0.45, n_estimators=150, max_depth=3).fit(x_train, y_train)
print(xg.score(x_train, y_train))
print(xg.score(x_test, y_test))

[15:16:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9726383686073803
0.7925462975863411


In [ ]:
x_pred= xg.predict(tftest_sc)

In [ ]:
fol = pd.DataFrame(x_pred, columns=['prediction'], index= None)
fol.set_index('prediction', inplace= True)
fol.to_csv('pred2.csv')

In [ ]:
fol.head()

In [18]:
#############################
tymm = MinMaxScaler()
tf_mm= tymm.fit_transform(tf)
tftest_mm = tymm.transform(tftest)

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tf_mm, y_, random_state=42)

In [50]:
xghp = XGBRegressor(colsample_bytree= 1, learning_rate= 0.25, max_depth= 3, n_estimators= 2000, subsample= 1).fit(x_train, y_train)
print(xghp.score(x_train, y_train))
print(xghp.score(x_test, y_test))

[06:43:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9999960711601398
0.8048213636255216


In [58]:
udil= xghp.predict(tftest_mm)

In [59]:
fol2 = pd.DataFrame(udil, columns=['prediction'], index= None)
fol2.set_index('prediction', inplace= True)
fol2.to_csv('pred2.csv')